The objective of this demo is to showcase a scenario in which a ficticious open source community wants to implement an AI system to assist the pull request analysis before they are merged or rejected. New pull requests should be classified as potentially acceptable or not, based on various features of the pull request. The AI system should be trustable, hence they want to use TrustML to assess the trustworthiness of the candidate classification models before their deployment.

We use a dataset from the MSR 2020 conference (https://github.com/zhangxunhui/new_pullreq_msr2020) to simulate the scenario and the TrustML package to evaluate and assess the model's trustworthiness. In this case, the dataset is considerably large (2.2 GB) and needs some preprocessing before it may be used to build a classification model on. The applied preprocessing is shown separately in the file "original_dataset_preprocessing.py".

# Step 1: Defining the configuration file
In this scenario, the open source community is interested in having an AI model complying with several trustworthiness criteria. In decreasing order of importance for them, the AI system should comply with:
1. Performance: It is important to correctly predict the potential acceptable pull requests.
2. Explainability: The community will need models that provide explanations to them, in order to enable the comprehension of how the AI system predicts the potential acceptability of the pull request.
3. Uncertainty: Knowing how uncertain the prediction is might influence the banker to (not) assume the risks of merging the pull requests.
3. Fairness (equal importance): Ethical aspects to prevent scandal. Protected attributes: gender of the pull request submitter.

Based on this, we specify a configuration file based on metrics belonging to the listed trustworthiness dimensions, and we specify the assessment method as a weighted average with equal weights for the two dimensions and the metrics that will be used.

This is the content of the configuration file we will use:

```yaml
metrics:
    - AccuracySKL
    - PrecisionSKL:
        multiclass_average: "binary"
    - RecallSKL:
        multiclass_average: "binary"
    - PPercentageSKL:
        protected_attributes: [contrib_gender]
        positive_class: 1
    - EqualOpportunitySKL:
        protected_attributes: [contrib_gender]
        positive_class: 1
    - FaithfulnessLIMESKL:
        explainer_path: "demos/pull_request/lime_explainer"
    - InvertedExpectedCalibrationSKL
    - InvertedBrierSKL
assessment_method:
    WeightedAverage:
        performance-0.5:
            AccuracySKL: 0.7
            PrecisionSKL: 0.15
            RecallSKL: 0.15
        uncertainty-0.15:                  
            InvertedBrierSKL: 0.5
            InvertedExpectedCalibrationSKL: 0.5
        explainability-0.2:                
            FaithfulnessLIMESKL: 1
        fairness-0.15:             
            PPercentageSKL: 0.5
            EqualOpportunitySKL: 0.5
```

# Step 2: Import relevant packages
The first step will consist in importing the TrustML package, the classification model that we will use in the demo (RandomForestClassifier) and some supporting functions/modules, notably pandas for the dataset loading/manipulation, LimeTabularExplainer to train and evaluate an explainer for the model, and train_test_split to partition the dataset.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from dill import dump
from lime.lime_tabular import LimeTabularExplainer
from trustML.TrustworthinessComputationFacane.TrustworthinessComputationFacane import TrustworthinessComputationFacane

pd.reset_option("max_columns")
pd.set_option('display.max_columns', None)

# Step 3: Load dataset and create train/test splits
Now we load the CSV file, extract the target column (i.e., if the pull request was merged or not), and split the dataset into training and test, with a 80%-20% proportion.

In [14]:

demo_path = './'
path_configuration = demo_path + 'config_pull_request.yml'
file_dataset = demo_path + 'new_pullreq-red.csv'

# Load the data
dataset = pd.read_csv(file_dataset, sep=",", header=0)
dataset.head(0)

# Extract target column
Y = dataset["merged_or_not"]
Y.describe()

# Decompose the dataset: Training and test split and drop target
X_train, X_test, Y_train, Y_test = train_test_split(dataset.drop(columns=["merged_or_not"]), Y, test_size=0.2, stratify=dataset.merged_or_not, random_state=1)

# Step 4: Create and train the classifier
We will now train a random forest classifier on the training set, as well as a lime tabular explainer that will be able to provide prediction's explanations based on the random forest's output. This explainer will be also assessed for trustworthiness as part of the explainability-related metrics.

In [15]:

# TRAIN A RANDOM FOREST CLASSIFIER
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, Y_train)

# TRAIN A LIME TABULAR EXPLAINER
lime_explainer = LimeTabularExplainer(X_train.values, feature_names=X_train.columns.values, class_names=[1,0])

# Step 5: Store required assets
The explainability-related metric *FaithfulnessLIMESKL* requires an additional properties, a LIME tabular explainer. Therefore, we will store such asset into the directory we have set in the configuration file. The "explainer_path" property corresponds to the previously fitted lime tabular explainer. We store it using dill.

In [16]:
# SAVE EXPLAINER TO THE DEMO DIRECTORY (OPTIONAL, ONLY NECESSARY IF THEY ARE NOT ALREADY PRESENT)
with open(demo_path + 'lime_explainer', 'wb') as explainer_file:
    dump(lime_explainer, explainer_file)

# Step 6: Compute the trustworthiness
Once trained, we will assess its trustworthiness with the TrustML package. For this, we instantiate a TrustworthinessComputationFacane, we call the loadTrustworthinessIndicator method with the path to the configuration file we specified, and lastly we call the computeTrustworthinessScore function, passing the trained model and the test dataset (features and target) to evaluate the model's trustworthiness in such dataset. This function stores the trust assessment as a JSON-formatted string.

In [17]:
# TRUST STUFF
trustpr = TrustworthinessComputationFacane()
trustpr.loadTrustworthinessIndicator(configPath=path_configuration)
trustpr.computeTrustworthinessScore(trainedModel=rf_classifier, dataX=X_test, dataY=Y_test)

Computing p-percentage vector...
Computing equal opportunity metric...
Computing faithfulness metric with LIME...


KeyboardInterrupt: 

Now we can print the complete trustworthiness assessment as a JSON-formatted string using the getTrustworthinessScore function:

In [ ]:
print(trustpr.getTrustworthinessScore())

Which results in the following output:

```javascript
{
  "name": "Trust",
  "weighted_score": 0.8,
  "children": [
    {
      "name": "performance",
      "weight": 0.5,
      "weighted_score": 0.39,
      "raw_score": 0.78,
      "children": [
        {
          "name": "AccuracySKL",
          "weight": 0.7,
          "weighted_score": 0.55,
          "raw_score": 0.78
        },
        {
          "name": "PrecisionSKL",
          "weight": 0.15,
          "weighted_score": 0.12,
          "raw_score": 0.78
        },
        {
          "name": "RecallSKL",
          "weight": 0.15,
          "weighted_score": 0.12,
          "raw_score": 0.79
        }
      ]
    },
    {
      "name": "uncertainty",
      "weight": 0.15,
      "weighted_score": 0.1,
      "raw_score": 0.65,
      "children": [
        {
          "name": "InvertedBrierSKL",
          "weight": 0.5,
          "weighted_score": 0.2,
          "raw_score": 0.41
        },
        {
          "name": "InvertedExpectedCalibrationSKL",
          "weight": 0.5,
          "weighted_score": 0.45,
          "raw_score": 0.9
        }
      ]
    },
    {
      "name": "fairness",
      "weight": 0.35,
      "weighted_score": 0.32,
      "raw_score": 0.9,
      "children": [
        {
          "name": "PPercentageSKL",
          "weight": 0.5,
          "weighted_score": 0.46,
          "raw_score": 0.92
        },
        {
          "name": "EqualOpportunitySKL",
          "weight": 0.5,
          "weighted_score": 0.44,
          "raw_score": 0.88
        }
      ]
    }
  ]
}
```

# Conclusions
Which results in a value of 0.8 of the trustworthiness indicator for the classification model that will conform the AI system.

This notebook has illustrated how easy it is to use the TrustML package to evaluate the trustworthiness of a classification model intended to be used as part of an AI system. In this case, the TrustML package has been used as part of a model building pipeline, obtaining a trustworthiness assessment of 0.8 (out of 1). According to the trustability criteria of the ficticious open source community, the model would be deemed acceptable, as their acceptance threshold is indeed 0.8.